In [20]:
import os.path
import requests
import pandas as pd
import textwrap

def generate_sparql_query(fullName, property_labels_to_ids, language='en', qid=None):
    """
    Query WikiData for the properties of the given person listed in the given property map,
    either by fullName or QID. When a QID is provided, ?itemLabel is not included in the query.
    :param fullName: Name of the person to query
    :param property_labels_to_ids: Dictionary mapping property labels to WikiData property IDs
    :param language: Language code for the query results
    :param qid: WikiData entity ID (QID) for the person
    :return: SPARQL query string
    """
    propSelection = ""
    for label, pid in property_labels_to_ids.items():
        if label.endswith("_id") or label.startswith("image") or label.startswith("date"):
            propSelection += f"""
                OPTIONAL {{ ?item wdt:{pid} ?{label}. }}"""
        else:
            propSelection += f"""
                OPTIONAL {{ ?item wdt:{pid} ?{label}Id . ?{label}Id rdfs:label ?{label} FILTER(LANG(?{label}) = "{language}") . }}"""

    if qid:
        selectClause = "SELECT DISTINCT ?item"
        itemConstraint = f"BIND(wd:{qid} AS ?item)."
        groupByClause = "GROUP BY ?item"
    else:
        selectClause = "SELECT DISTINCT ?item ?itemLabel"
        selectClause += "\n    ".join([f"(SAMPLE(?{label}) AS ?{label})" for label in property_labels_to_ids.keys()])
        itemConstraint = f'?item wdt:P31 wd:Q5; rdfs:label "{fullName}"@{language} .'
        groupByClause = "GROUP BY ?item ?itemLabel"
    
    query = textwrap.dedent(f"""
    {selectClause} 
    WHERE {{
        {itemConstraint}
        {propSelection}
    }}
    {groupByClause}
    """)

    return query



def construct_image_url(filename):
    return f"https://commons.wikimedia.org/wiki/Special:FilePath/{requests.utils.quote(filename)}"



def query_wikidata(fullName, property_map, language='en', qid=None):
    SPARQL_ENDPOINT = "https://query.wikidata.org/sparql"
    query = generate_sparql_query(fullName, property_map, language, qid=qid)
    headers = {'User-Agent': 'Mozilla/5.0', 'Accept': 'application/json'}
    response = requests.get(SPARQL_ENDPOINT, headers=headers, params={'query': query, 'format': 'json'})

    if response.status_code != 200:
        print(query)
        response.raise_for_status()
        
    results = response.json()['results']['bindings']
    
    if not results:
        return []

    for i, result in enumerate(results):
        # Initialize with fullName to ensure it appears first
        data = {
            'fullName': fullName
        }
   
        for label in property_map:
            if label in result:
                value = result[label]['value']
                data[label] = value
            else:
                data[label] = None
                
        # add item URI
        data['qid'] = os.path.basename(result['item']['value'])
        results[i] = data
            
    return results

def get_wikipedia_links(qid, languages):
    """
    Fetch Wikipedia links for a given Wikidata QID and a list of languages.

    Parameters:
    - qid (str): The QID of the Wikidata item.
    - languages (list): A list of language codes (e.g., ['en', 'de']).

    Returns:
    - dict: A dictionary with languages as keys and Wikipedia URLs as values.
    """
    url = "https://www.wikidata.org/w/api.php"
    params = {
        "action": "wbgetentities",
        "ids": qid,
        "props": "sitelinks/urls",
        "format": "json"
    }

    response = requests.get(url, params=params)
    data = response.json()

    links = {}
    if "entities" in data and qid in data["entities"]:
        sitelinks = data["entities"][qid].get("sitelinks", {})
        for lang in languages:
            sitekey = f"{lang}wiki"
            if sitekey in sitelinks:
                # Use the 'title' key and construct the URL manually
                title = sitelinks[sitekey]["title"]
                links[lang] = f"https://{lang}.wikipedia.org/wiki/{requests.utils.quote(title)}"
            else:
                links[lang] = None  # Or use '' to represent absence of link

    return links


def get_person_info_from_wikidata(names, property_map, languages=None):
    if languages is None:
        languages = ['en', 'de']
    all_data = []
    print('Retrieving scholar data...')
    for item in names:
        if type(item) is tuple:
            results = query_wikidata(item[0], property_map, languages[0], qid=item[1])
        else:
            results = query_wikidata(item, property_map, languages[0])
        
        all_data += results
    if len(all_data) > 0:
        # Ensure fullName appears first by reordering columns based on property_labels_to_ids keys
        columns_order = ['fullName', 'qid'] + list(property_map.keys())
        df = pd.DataFrame(all_data, columns=columns_order)
        
        # Add wikipedia links
        print("Retrieving wikipedia URLs")
        for language in languages:
            df[f'wikipedia_{language}'] = df.apply(lambda r: get_wikipedia_links(r['qid'], language).get(language), axis=1)
    else:
        df = pd.DataFrame(columns=['fullName'] + list(property_map.keys()) + [f'wikipedia_{language}'])
    return df

In [22]:
# Now calling the updated function with the 'language' parameter
property_labels_to_ids = {
    'sexOrGender': 'P21',
#    'image': 'P18',
    'countryOfCitizenship': 'P27',
    'givenName': 'P735',
    'familyName': 'P734',
    'dateOfBirth': 'P569',
    'dateOfDeath': 'P570',
    'occupation': 'P106',
    'fieldOfWork': 'P101',
    'viaf_id': 'P214',
    'isni_id': 'P213',
    'gnd_id': 'P227'
}

scholars = [
    "Hans Kelsen",
    "Hugo Sinzheimer",
    ("Karl Renner","Q11726"),
    ("Ernst Fraenkel", "Q86812"),
    ("Franz Leopold Neumann", "Q63195"),
    "Otto Kahn-Freund",
    "Otto Kirchheimer",
    "Herrmann Kantorowicz",
    ("Ludwig Bendix", "Q28053205"),
    ("Arthur Nussbaum", "Q103088"),
    "Theodor Geiger",
    "Erhard Blankenburg",
    "Wolfgang Kaupen",
    "Rüdiger Lautmann",
    "Thilo Ramm",
    "Rudolf Wiethölter",
    ("Niklas Luhmann","Q57238"),
    "Gunther Teubner",
    "Volkmar Gessner",
    "Konstanze Plett",
    "Ute Sacksofsky",
    ("Susanne Baer","Q101872")
]
df = get_person_info_from_wikidata(scholars, property_labels_to_ids)
df

Retrieving scholar data...


ConnectionError: HTTPSConnectionPool(host='query.wikidata.org', port=443): Max retries exceeded with url: /sparql?query=%0ASELECT+DISTINCT+%3Fitem+%3FitemLabel%28SAMPLE%28%3FsexOrGender%29+AS+%3FsexOrGender%29%0A%28SAMPLE%28%3FcountryOfCitizenship%29+AS+%3FcountryOfCitizenship%29%0A%28SAMPLE%28%3FgivenName%29+AS+%3FgivenName%29%0A%28SAMPLE%28%3FfamilyName%29+AS+%3FfamilyName%29%0A%28SAMPLE%28%3FdateOfBirth%29+AS+%3FdateOfBirth%29%0A%28SAMPLE%28%3FdateOfDeath%29+AS+%3FdateOfDeath%29%0A%28SAMPLE%28%3Foccupation%29+AS+%3Foccupation%29%0A%28SAMPLE%28%3FfieldOfWork%29+AS+%3FfieldOfWork%29%0A%28SAMPLE%28%3Fviaf_id%29+AS+%3Fviaf_id%29%0A%28SAMPLE%28%3Fisni_id%29+AS+%3Fisni_id%29%0A%28SAMPLE%28%3Fgnd_id%29+AS+%3Fgnd_id%29+%0AWHERE+%7B%0A++++%3Fitem+wdt%3AP31+wd%3AQ5%3B+rdfs%3Alabel+%22Theodor+Geiger%22%40en+.%0A%0A++++++++++++OPTIONAL+%7B+%3Fitem+wdt%3AP21+%3FsexOrGenderId+.+%3FsexOrGenderId+rdfs%3Alabel+%3FsexOrGender+FILTER%28LANG%28%3FsexOrGender%29+%3D+%22en%22%29+.+%7D%0A++++++++++++OPTIO

In [ ]:
from lib.dnb_data import get_publications

In [3]:
df.to_csv("scholars.csv", index=False)

NameError: name 'df' is not defined

In [4]:
import pandas as pd

# Load the data
df = pd.read_csv("scholars.csv", encoding='utf-8')
# Convert date strings to datetime
df['dateOfBirth'] = pd.to_datetime(df['dateOfBirth'])
df['dateOfDeath'] = pd.to_datetime(df['dateOfDeath'])

# Create a new column for the duration
df['duration'] = df['dateOfDeath'] - df['dateOfBirth']

# Display the first few rows to check
df.head()


,fullName,item,sexOrGender,image,countryOfCitizenship,givenName,familyName,dateOfBirth,dateOfDeath,occupation,fieldOfWork,employer,viaf_id,isni_id,gnd_id,duration
0,Hans Kelsen,Q84165,male,http://commons.wikimedia.org/wiki/Special:File...,Cisleithania,Hans,Kelsen,1881-10-11 00:00:00+00:00,1973-04-19 00:00:00+00:00,judge,international law,Charles University,31998356,0000000121266076,118561219.0,33427 days
1,Hugo Sinzheimer,Q86043,male,http://commons.wikimedia.org/wiki/Special:File...,Germany,Hugo,Sinzheimer,1875-04-12 00:00:00+00:00,1945-09-16 00:00:00+00:00,lawyer,NaN,Goethe University Frankfurt,27864307,0000000109619641,118614711.0,25724 days
2,Karl Renner,Q11726,male,http://commons.wikimedia.org/wiki/Special:File...,Cisleithania,Karl,Renner,1870-12-14 00:00:00+00:00,1950-12-31 00:00:00+00:00,lawyer,politics,Austrian Federal Government,61669459,0000000121358165,118599739.0,29236 days
3,Ernst Fraenkel,Q86812,male,NaN,Germany,Ernst,Fraenkel,1898-12-26 00:00:00+00:00,1975-03-28 00:00:00+00:00,lawyer,NaN,Free University Berlin,27108403,0000000110230959,118534602.0,27850 days
4,Franz Leopold Neumann,Q112562068,male,NaN,NaN,Leopold,Neumann,NaT,NaT,printer,publishing,NaN,637163874508945722514,NaN,NaN,NaT


In [8]:
import pandas as pd
import plotly.express as px

# Assuming df is already loaded and dateOfBirth/dateOfDeath are converted to datetime

# Initialize a list to track the last dateOfDeath in each row to manage overlaps
last_dates = []

# Function to find the appropriate row for each scholar
def find_row(last_dates, start_date):
    for i, last_date in enumerate(last_dates):
        if start_date > last_date:
            return i
    return len(last_dates)

# Assign rows without overlaps and sort by the earliest dateOfBirth
df['row'] = 0
for index, scholar in df.iterrows():
    row = find_row(last_dates, scholar['dateOfBirth'])
    if row < len(last_dates):
        last_dates[row] = scholar['dateOfDeath']
    else:
        last_dates.append(scholar['dateOfDeath'])
    df.at[index, 'row'] = row

# Now plotting without row labels
fig = px.timeline(df, x_start="dateOfBirth", x_end="dateOfDeath", y="row", text="fullName", title="Scholars' Life Spans Timeline")

# Update layout
fig.update_layout(yaxis=dict(tickmode='array', tickvals=[], ticktext=[]))
fig.update_yaxes(autorange="reversed")  # This reverses the y-axis to match your requirement

fig.show()



In [44]:
import pandas as pd
from datetime import datetime

# Assuming df is your existing DataFrame

# Convert dateOfBirth and dateOfDeath to just the year, handle NaT/NaN appropriately
df['Year'] = pd.to_datetime(df['dateOfBirth'], errors='coerce').dt.year.astype('Int64')
df['End Year'] = pd.to_datetime(df['dateOfDeath'], errors='coerce').dt.year.astype('Int64')

# Create 'Display Date' as "dateOfBirth - dateOfDeath"
df['Display Date'] = df['Year'].astype(str).replace('<NA>','')  + ' - ' + df['End Year'].astype(str).replace('<NA>','')

# Create 'Headline' as "fullName (dateOfBirth - dateOfDeath)"
df['Headline'] = df['fullName'] + ' (' + df['Display Date'] + ')'

# Create 'Text' column by combining occupation, fieldOfWork, employer
df['Text'] = df[['occupation', 'fieldOfWork']].apply(lambda x: '<br>'.join(x.dropna()), axis=1)

# Use the image directly; assuming the URLs are already correctly formed in the 'image' column
df['Media'] = df['image']

# Add a "Group" column with the value "actors" for all rows
df['Group'] = 'actors'

# fix date columns
df['Display Date'] = df['Display Date'].fillna('')  # Ensure no NaNs in Display Date
df['Headline'] = df['Headline'].fillna('')  # Ensure no NaNs in Headline
df['Text'] = df['Text'].fillna('')  # Ensure no NaNs in Text
df['Media'] = df['Media'].fillna('')  # Ensure no NaNs in Media

# Now select and order the DataFrame according to the TimelineJS template requirements
columns = "Year	Month	Day	Time	End Year	End Month	End Day	End Time	Display Date	Headline	Text	Media	Media Credit	Media Caption	Media Thumbnail	Type	Group	Background	Link".split("\t")
for col in columns:
    if col not in df:
        df[col] = ''
timeline_df = df[columns]

timeline_df.to_excel("timeline_data.xlsx", index=False)
